In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import shutil

In [25]:
folders = ['64', '64-bw', '64-gray', '256', '256-gray', '256-bw', '128', '128-gray', '128-bw']

for a in folders:
    i=0
    for filename in os.listdir(f'Hey-Waldo/{a}/waldo'):
        dst =a + str(i) + ".jpg"
        src =f'Hey-Waldo/{a}/waldo/'+ filename 
        dst =f'Hey-Waldo/{a}/waldo/'+ dst 
          
        # rename() function will 
        # rename all the files 
        os.rename(src, dst) 
        i += 1
    

In [26]:
for a in folders:
    i=0
    for filename in os.listdir(f'Hey-Waldo/{a}/notwaldo'):
        dst =a + str(i) + ".jpg"
        src =f'Hey-Waldo/{a}/notwaldo/'+ filename 
        dst =f'Hey-Waldo/{a}/notwaldo/'+ dst 
          
        # rename() function will 
        # rename all the files 
        os.rename(src, dst) 
        i += 1

In [36]:
# Creating Train / Val / Test folders (One time use)
root_dir = 'Hey-Waldo/waldoAll'
classes_dir = ['/waldo', '/notwaldo']

val_ratio = 0.15
test_ratio = 0.05

for cls in classes_dir:
    os.makedirs(root_dir +'/train' + cls)
    os.makedirs(root_dir +'/val' + cls)
    os.makedirs(root_dir +'/test' + cls)
    # Creating partitions of the data after shuffeling
    src = root_dir + cls # Folder to copy images from

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - val_ratio + test_ratio)), 
                                                               int(len(allFileNames)* (1 - test_ratio))])


    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    print('Testing: ', len(test_FileNames))

    # Copy-pasting images
    for name in train_FileNames:
        if name[-1]=='g':
            shutil.copy(name, root_dir +'/train' + cls)

    for name in val_FileNames:
        if name[-1]=='g':
            shutil.copy(name, root_dir +'/val' + cls)

    for name in test_FileNames:
        if name[-1]=='g':
            shutil.copy(name, root_dir +'/test' + cls)

Total images:  269
Training:  242
Validation:  13
Testing:  14
Total images:  20528
Training:  18475
Validation:  1026
Testing:  1027
